# 02 — Feature Engineering

This notebook prepares a **clean, information-rich feature matrix** for churn modeling. I will transform raw fields into signals that reflect customer behavior, reduce noise (missing data, skew, outliers), and make the dataset **model-ready and reproducible**.

## Objectives
- Convert raw columns into **predictive, interpretable** features.
- Handle **missing data**, **skewness**, and **outliers** safely.
- Encode categorical variables and **scale** numerics in a robust way.
- Build a **scikit-learn Pipeline** (idempotent & versionable) to reproduce the same features in training and inference.
- Export the final matrix to `data/processed/` and persist the pipeline to `artifacts/` for the modeling step.

## Design Principles
- **No leakage:** compute features using only information available at prediction time.
- **Simple → Strong:** start with well-founded transforms (imputation, scaling, encoding) and add domain signals (rates, bands, RFM).
- **Reproducibility:** all steps wrapped in a `Pipeline`/`ColumnTransformer` with fixed random seeds.
- **Explainability:** favor features that are easy to interpret and act on (rates, flags, bands).

## Techniques & Tools
- **Imputation**
  - Numeric: median
  - Categorical: constant “Missing”
- **Scaling**
  - `RobustScaler` (resistant to outliers)
- **Encoding**
  - One-Hot for low/medium cardinality
  - (Optional later) Target/Count encoding for high cardinality if needed
- **Transformations**
  - Log/yeo-johnson or **binning** for highly skewed variables
- **Engineered Features (examples)**
  - `orders_per_month = OrderCount / (Tenure + 1)`
  - `engagement_rate = HourSpendOnApp / (Tenure + 1)`
  - `coupon_rate = CouponUsed / (OrderCount + 1)`
  - `cashback_per_order = CashbackAmount / (OrderCount + 1)`
  - `high_growth_flag = (OrderAmountHikeFromlastYear ≥ 18)`
  - `recency_band` from `DaySinceLastOrder` (0–2, 3–7, 8–14, 15+)
  - `tenure_band` from `Tenure` (0–3, 4–12, 13–24, 25+)
- **Infrastructure**
  - `scikit-learn` Pipelines + `ColumnTransformer`
  - Outputs: Parquet features + Joblib pipeline

## Inputs
- **Raw**: `data/raw/E_Commerce_Dataset.xlsx` (sheet: “E Comm”)
- **Columns of interest**
  - **Numeric:** Tenure, HourSpendOnApp, OrderCount, CashbackAmount, SatisfactionScore, DaySinceLastOrder, CouponUsed, OrderAmountHikeFromlastYear, NumberOfDeviceRegistered, NumberOfAddress
  - **Categorical:** Gender, MaritalStatus, PreferredPaymentMode, PreferredLoginDevice, CityTier, PreferedOrderCat, Complain
  - **Target:** Churn

## Outputs
- `data/processed/churn_features.parquet` — model-ready matrix
- `artifacts/feature_pipeline.joblib` — fitted pipeline for reuse in training/inference

## Quality Checks
- Feature matrix **shape** and **schema** (names, dtypes)
- Missing values **after** preprocessing (expect none)
- Basic drift/summary stats of engineered features (sanity)
- Exported **feature list** for modeling

> Next: implement the pipeline (impute → engineer → encode/scale), save artifacts, and validate the final matrix before moving to `03_modeling.ipynb`.


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
import joblib


In [2]:
# If this notebook lives in churn_analysis/notebooks/, parent points to project root
project_root = Path.cwd().parent

# IO paths
raw_path   = project_root / "data" / "raw" / "E_Commerce_Dataset.xlsx"
proc_path  = project_root / "data" / "processed" / "churn_features.parquet"
pipe_path  = project_root / "artifacts" / "feature_pipeline.joblib"

# Create output folders
proc_path.parent.mkdir(parents=True, exist_ok=True)
pipe_path.parent.mkdir(parents=True, exist_ok=True)

print(f"[INFO] project_root = {project_root}")
print(f"[INFO] raw_path     = {raw_path}")

[INFO] project_root = c:\Users\luizo\Projetos\churn_analysis
[INFO] raw_path     = c:\Users\luizo\Projetos\churn_analysis\data\raw\E_Commerce_Dataset.xlsx


# Load raw data & basic cleaning

In [4]:
# Read specific sheet and remove unnamed/empty columns
df = pd.read_excel(raw_path, sheet_name="E Comm")
df = df.loc[:, ~df.columns.str.match(r"^Unnamed")].copy()

print(f"[INFO] Loaded: {df.shape[0]} rows x {df.shape[1]} cols")
df.head()

[INFO] Loaded: 5630 rows x 20 cols


,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60


# Define column groups & run validations

In [5]:
target_col = "Churn"

num_cols = [
    "Tenure", "HourSpendOnApp", "OrderCount", "CashbackAmount",
    "SatisfactionScore", "DaySinceLastOrder", "CouponUsed",
    "OrderAmountHikeFromlastYear", "NumberOfDeviceRegistered",
    "NumberOfAddress"
]

cat_cols = [
    "Gender", "MaritalStatus", "PreferredPaymentMode",
    "PreferredLoginDevice", "CityTier", "PreferedOrderCat", "Complain"
]

# --- Guard rails: verify presence of all columns ---
missing_cols = [c for c in (num_cols + cat_cols + [target_col]) if c not in df.columns]
if missing_cols:
    raise ValueError(f"[ERROR] Columns not found in dataframe: {missing_cols}")

# --- Split features/target ---
X = df[num_cols + cat_cols].copy()
y = df[target_col].astype(int)

# --- Ensure numeric dtypes (coerce silent non-numerics to NaN) ---
X[num_cols] = X[num_cols].apply(pd.to_numeric, errors="coerce")

# --- Basic sanity checks ---
# Target must be binary and non-null
uniq_target = set(y.dropna().unique())
assert uniq_target <= {0, 1}, f"[ERROR] Churn must be binary, got: {uniq_target}"
print(f"[CHECK] Target unique values: {sorted(list(uniq_target))}")

# CustomerID uniqueness (if present)
if "CustomerID" in df.columns:
    dup_ids = df["CustomerID"].duplicated().sum()
    print(f"[CHECK] Duplicated CustomerID: {dup_ids}")

# Missing summary for numerics and categoricals
num_missing = X[num_cols].isna().sum().sort_values(ascending=False)
cat_missing = X[cat_cols].isna().sum().sort_values(ascending=False)

print("\n[INFO] Missing (numeric):")
print(num_missing[num_missing > 0].to_string())

print("\n[INFO] Missing (categorical):")
print(cat_missing[cat_missing > 0].to_string())

print(f"\n[INFO] Features shape: {X.shape} | Target shape: {y.shape}")

[CHECK] Target unique values: [np.int64(0), np.int64(1)]
[CHECK] Duplicated CustomerID: 0

[INFO] Missing (numeric):
DaySinceLastOrder              307
OrderAmountHikeFromlastYear    265
Tenure                         264
OrderCount                     258
CouponUsed                     256
HourSpendOnApp                 255

[INFO] Missing (categorical):
Series([], )

[INFO] Features shape: (5630, 17) | Target shape: (5630,)


# Feature engineering (preview only — no pipeline yet)

In [7]:
def add_engineered_features(X_df: pd.DataFrame)-> pd.DataFrame:
    X_en= X_df.copy()

    # --- Rate features (behavioral intensities) ---
    X_en["orders_per_month"]   = X_en["OrderCount"]       / (X_en["Tenure"] + 1)
    X_en["engagement_rate"]    = X_en["HourSpendOnApp"]   / (X_en["Tenure"] + 1)
    X_en["coupon_rate"]        = X_en["CouponUsed"]       / (X_en["OrderCount"] + 1)
    X_en["cashback_per_order"] = X_en["CashbackAmount"]   / (X_en["OrderCount"] + 1)

    # --- Growth flag (YoY high tier) ---
    X_en["high_growth_flag"] = (X_en["OrderAmountHikeFromlastYear"] >= 18).astype(int)

    # --- Recency band (non-linear effect buckets) ---
    rec_bins   = [-np.inf, 2, 7, 14, np.inf]
    rec_labels = ["0-2", "3-7", "8-14", "15+"]
    X_en["recency_band"] = pd.cut(X_en["DaySinceLastOrder"], bins=rec_bins, labels=rec_labels, ordered=True)

    # --- Tenure band (lifecycle stage) ---
    ten_bins   = [-np.inf, 3, 12, 24, np.inf]
    ten_labels = ["0-3", "4-12", "13-24", "25+"]
    X_en["tenure_band"] = pd.cut(X_en["Tenure"], bins=ten_bins, labels=ten_labels, ordered=True)

    return X_en

# Create transformer (will be used later inside the pipeline)
engineer = FunctionTransformer(add_engineered_features, validate=False)

# --- Preview engineered features (no pipeline yet) ---
X_eng_preview = add_engineered_features(X)

eng_num = ["orders_per_month", "engagement_rate", "coupon_rate", "cashback_per_order"]
eng_cat = ["recency_band", "tenure_band", "high_growth_flag"]  # high_growth_flag is numeric but preview here too

print("[INFO] Engineered numeric features (head):")
display(X_eng_preview[eng_num].head(5))

print("\n[INFO] Engineered categorical features (head):")
display(X_eng_preview[eng_cat].head(5))

# Quick NA check on engineered columns
na_counts_eng = X_eng_preview[eng_num + ["recency_band", "tenure_band"]].isna().sum().sort_values(ascending=False)
print("\n[CHECK] Missing values in engineered features:")
print(na_counts_eng.to_string())

# Sanity checks: ranges should be non-negative for rate features
for c in eng_num:
    min_val = X_eng_preview[c].min(skipna=True)
    assert min_val >= 0, f"[ERROR] {c} has negative values (min={min_val})"
print("\n[CHECK] Non-negativity passed for rate features.")

[INFO] Engineered numeric features (head):


,orders_per_month,engagement_rate,coupon_rate,cashback_per_order
0,0.2,0.6,0.5,79.965
1,NaN,NaN,0.0,60.450
2,NaN,NaN,0.0,60.140
3,1.0,2.0,0.0,67.035
4,1.0,NaN,0.5,64.800



[INFO] Engineered categorical features (head):


,recency_band,tenure_band,high_growth_flag
0,3-7,4-12,0
1,0-2,NaN,0
2,3-7,NaN,0
3,3-7,0-3,1
4,3-7,0-3,0



[CHECK] Missing values in engineered features:
orders_per_month      522
engagement_rate       519
coupon_rate           514
recency_band          307
tenure_band           264
cashback_per_order    258

[CHECK] Non-negativity passed for rate features.
